In [8]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize,word_tokenize
import numpy as np
import math
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
def read_article(para):
    file = open( r"C:\Users\user\vijay.txt",encoding='utf-8')
    filedata = file.read()
#     print(filedata)
    sentences = sent_tokenize(filedata)
    #print("Tokenized Sentences: ",sentences)
    sent=word_tokenize(filedata)

    return sentences
file = open( r"C:\Users\user\vijay.txt",encoding='utf-8')
filedata = file.read()
sentences = sent_tokenize(filedata)
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
    
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    #build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
    #print(vector1)
    file1=open("myOutFile.txt",'a')
    file1.write(str(vector1))
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
    # print(cosine_distance(vector1,vector2))
    return cosine_distance(vector1, vector2)
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    #print(similarity_matrix)
    return similarity_matrix

def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    #ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        #print(words)
        for word in words:
            word = word.lower()
            #word = ps.stem(word)
            #print(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table
#         print(frequency_matrix)
    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
         
        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))
            
        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1
#     print(word_per_doc_table)
    return word_per_doc_table

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    sum=0
    text=[]
    
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)
#             print(tf_idf_table[word1])
        tf_idf_matrix[sent1]= tf_idf_table
        s=len(tf_idf_matrix[sent1])
#         print(tf_idf_matrix[sent1])
        for i in tf_idf_matrix[sent1].values():
#         sum=sum+tf_idf_table[word1]
#        
            sum=sum+i
        text.append(sum/s)   
#         print("total sum",sum)
        sum=0
    print(text)
    return tf_idf_matrix

def generate_summary(para, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(para)
    
    total_documents = len(sentences)
    
    freq_matrix = _create_frequency_matrix(sentences)
    #print("freqency_matrix",freq_matrix)

    
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print("tf_matrix:",tf_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print("idf_matrix",idf_matrix)

    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print("\n\n")
#     print("tf-idf: ",tf_idf_matrix)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    
    
# Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(sentences)


# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=[sentences])
    df
    #print(df)
    
#     print(cosine_similarity(df, df))
    tent=[]
    text=_create_tf_idf_matrix(tf_matrix, idf_matrix)
    x=np.array(cosine_similarity(df, df))
    y=np.argwhere(x > 0.4)
    z=np.split(y,1,1)
    print(y)
    for i in range(len(y)):
        for j in range(len(y[i])):
            if(y[i][i]==y[i][j]):
                continue
            elif(text[1]>text[j]):
                tent.append(text[i])
            else:
                tent.append(text[j])
    print(tent)


   
   
generate_summary( "msft.txt", 2)
    
   
    
   
            
    
        

[0.03397657711995336, 0.07603363368871718, 0.04745215238980775, 0.02135610788939889, 0.04296075841588464, 0.03487413197655753, 0.15845588250267315]
[0.03397657711995336, 0.07603363368871718, 0.04745215238980775, 0.02135610788939889, 0.04296075841588464, 0.03487413197655753, 0.15845588250267315]
[[0 0]
 [0 5]
 [1 1]
 [2 2]
 [2 3]
 [3 2]
 [3 3]
 [3 4]
 [4 3]
 [4 4]
 [5 0]
 [5 5]
 [6 6]]


KeyError: 1